In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/인구밀도(2021-2022).csv', encoding='utf-8')
df = df[df['동별(3)'] == '소계']
df_population = df[['동별(2)', '2021', '2022']]
df_population.columns = ['자치구명', '인구_2021', '인구_2022']
df_population.drop(index=1, inplace=True)
df_population['인구_2021'] = df_population['인구_2021'].astype(int)
df_population['인구_2022'] = df_population['인구_2021'].astype(int)

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/882128261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population.drop(index=1, inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/882128261.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_population['인구_2021'] = df_population['인구_2021'].astype(int)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/882128261.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_index

In [3]:
df_elec = pd.read_csv('./data/ghgEmissions_data_electricity.csv', encoding='utf-8', skiprows=1)
df_elec = df_elec[['자치구명', '계']]
df_elec['계'] = df_elec['계'].str.replace(',', '').astype('float')
df_gas = pd.read_csv('./data/ghgEmissions_data_gas.csv', encoding='utf-8', skiprows=1)
df_gas = df_gas[['자치구명', '계']]
df_gas['계'] = df_gas['계'].str.replace(',', '').astype('float')
df_district_heating = pd.read_csv('./data/ghgEmissions_data_district_heating.csv', encoding='utf-8', skiprows=1)
df_district_heating = df_district_heating[['자치구명', '계']]
df_district_heating['계'] = df_district_heating['계'].str.replace(',', '').astype('float')
ghg_emissions = pd.merge(df_elec, df_gas, on='자치구명')
ghg_emissions = pd.merge(ghg_emissions, df_district_heating, on='자치구명')
ghg_emissions['온실가스배출량'] = ghg_emissions['계_x'] + ghg_emissions['계_y'] + ghg_emissions['계']
ghg_emissions.drop(['계_x', '계_y', '계'], axis=1, inplace=True)

In [4]:
env_index = pd.merge(df_population, ghg_emissions, on='자치구명')
env_index['온실가스(1인당)'] = env_index['온실가스배출량'] / env_index['인구_2022']
env_index.drop(['온실가스배출량'], axis=1, inplace=True)

In [5]:
df = pd.read_csv('./data/전력+사용량(용도별).csv', encoding='utf-8')
elec_consumption = df[['자치구별(2)', '2022']]
elec_consumption.drop(index=range(4), inplace=True)
elec_consumption.columns = ['자치구명', '전력사용량']

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/1242192353.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elec_consumption.drop(index=range(4), inplace=True)


In [6]:
env_index = pd.merge(env_index, elec_consumption, on='자치구명')

In [7]:
df = pd.read_csv('./data/서울특별시 공공자전거 대여소별 이용정보(월별)_23.1-6.csv', encoding='cp949')
df['대여건수'] = pd.to_numeric(df['대여건수'].str.replace(',', ''), errors='coerce')
df['반납건수'] = pd.to_numeric(df['반납건수'].str.replace(',', ''), errors='coerce')
bicycle_usage = df.groupby('자치구', as_index=False).agg({'대여건수': 'sum', '반납건수': 'sum'})
bicycle_usage.columns = ['자치구명', '자전거_대여건수', '자전거_반납건수']
bicycle_usage['공공자전거_이용건수'] = (bicycle_usage['자전거_대여건수'] + bicycle_usage['자전거_반납건수'])/2
bicycle_usage['공공자전거_이용건수'] = bicycle_usage['공공자전거_이용건수'].astype(int)
bicycle_usage.drop(['자전거_대여건수', '자전거_반납건수'], axis=1, inplace=True)

In [8]:
env_index = pd.merge(env_index, bicycle_usage, on='자치구명')

In [9]:
df = pd.read_csv('./data/급수+사용량+및+사용료(2012년+이후).csv', encoding='utf-8')
water_usage = df[['자치구별(2)', '2022']]
water_usage.columns = ['자치구명', '급수사용량']
water_usage.drop(index=range(4), inplace=True)
water_usage['급수사용량'] = water_usage['급수사용량'].astype('float')

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/19716127.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_usage.drop(index=range(4), inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/19716127.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  water_usage['급수사용량'] = water_usage['급수사용량'].astype('float')


In [10]:
env_index = pd.merge(env_index, water_usage, on='자치구명')

In [11]:
env_index['급수사용량(1인당)'] = env_index['급수사용량'] / env_index['인구_2022']
env_index.drop(['급수사용량'], axis=1, inplace=True)

In [12]:
waste_emission = pd.read_csv('./data/주민+1인당+생활폐기물(쓰레기)+배출량.csv', encoding='utf-8')
waste_emission.columns = ['자치구명', '쓰레기배출량(1인당)', '생활폐기물 배출량', '주민수']
waste_emission = waste_emission[['자치구명', '쓰레기배출량(1인당)']]
waste_emission.drop(range(2), inplace=True)

In [13]:
env_index = pd.merge(env_index, waste_emission, on='자치구명')

In [14]:
df = pd.read_csv('./data/대기오염(구별).csv', encoding='utf-8')
air_data = df[['구분별(1)']].join(df.filter(like='2022', axis=1))
air_data.drop(index=range(1,4), inplace=True)
air_data.columns = ['자치구명', '아황산가스', '일산화탄소', '이산화질소', '미세먼지', '초미세먼지', '오존']
air_data.drop(index=0, columns=['일산화탄소', '초미세먼지'], inplace=True)

In [15]:
env_index = pd.merge(env_index, air_data, on='자치구명')

In [16]:
df = pd.read_csv('./data/서울시_공원통계(2022).csv', encoding='utf-8')
park = df[['자치구별(2)', '2022.2']]
park.drop(index=range(5), inplace=True)
park.drop(index=30, inplace=True)
park.columns = ['자치구명', '공원면적']
park['공원면적'] = park['공원면적'].astype('float')

/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/2230141381.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park.drop(index=range(5), inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/2230141381.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  park.drop(index=30, inplace=True)
/var/folders/qr/d0tz05096cq7wtbf54br3qsw0000gn/T/ipykernel_42423/2230141381.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/p

In [17]:
env_index = pd.merge(env_index, park, on='자치구명')

In [18]:
env_index['공원면적(1인당)'] = env_index['공원면적'] / env_index['인구_2022']
env_index.drop(columns=['공원면적', '인구_2021', '인구_2022'], inplace=True)

In [19]:
env_index.head()

,자치구명,온실가스(1인당),전력사용량,공공자전거_이용건수,급수사용량(1인당),쓰레기배출량(1인당),아황산가스,이산화질소,미세먼지,오존,공원면적(1인당)
0,종로구,6.764409,1721240,726636,0.165591,1.49,0.003,0.021,32,0.031,0.074353
1,중구,10.138907,2305200,500302,0.206970,2.26,0.003,0.024,30,0.030,0.024329
2,용산구,3.827852,1514990,441899,0.115397,1.24,0.003,0.021,33,0.030,0.007476
3,성동구,4.072864,2235050,859819,0.107414,0.93,0.003,0.021,35,0.028,0.010537
4,광진구,2.736378,1616001,1042726,0.101943,0.71,0.003,0.019,31,0.027,0.009812


In [20]:
env_index.to_csv('./result/seoul_env_data.csv', encoding='utf-8-sig', index=False)